# Sychronous

In [1]:
"""All custom functions used in this app"""
import re
from typing import List

import datetime
from datetime import datetime as dt
from langchain_core.documents import Document
from sqlalchemy import create_engine, Column, String, Text, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base

from backend.app.build.models import LLM
from backend.app.utils.config import SYNC_DATABASE_URL


# Create the SQLAlchemy engine
engine = create_engine(SYNC_DATABASE_URL)

# Create a base class for declarative class definitions
Base = declarative_base()


class WikiSummary(Base):
    """Define the wiki_summaries table as a class"""
    __tablename__ = 'wiki_summaries'
    uuid = Column(String, primary_key=True)
    url = Column(String, nullable=False)
    summary = Column(Text, nullable=False)
    creation_date = Column(DateTime,
                           default=dt.now(datetime.timezone.utc))


# Create the table in the database
Base.metadata.create_all(engine)


# Create a session factory
session_constructor = sessionmaker(bind=engine)
session = session_constructor()


def insert_wiki_summary(uuid, url, summary):
    """
    Function creates a new record and inserts it into the table
    wiki_summaries.
    """
    try:
        # Create a new WikiSummary object
        new_summary = WikiSummary(
            uuid=uuid,
            url=url,
            summary=summary
        )

        # Add the new object to the session
        session.add(new_summary)

        # Commit the transaction
        session.commit()

        info = "Data inserted successfully!"

        return info

    except Exception as e:
        info = f"An error occurred: {e}"
        session.rollback()
        return info
    finally:
        # Close the session
        session.close()

I0000 00:00:1726895605.213336  362042 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1726895605.220067  362042 check_gcp_environment.cc:61] BIOS data file does not exist or cannot be opened.


In [3]:
insert_wiki_summary("123e4567-e89b-12d3-a456-426614174002", "https://example.com", "This is a summary.")

'Data inserted successfully!'

# Asynchronous

In [5]:
%pip install sqlalchemy[asyncio] asyncpg

I0000 00:00:1726894965.144419  348363 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1726894965.145276  348363 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import asyncio
import datetime

from datetime import datetime as dt
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, String, Text, DateTime


from backend.app.utils.config import ASYNC_DATABASE_URL


# Create the SQLAlchemy engine
engine = create_async_engine(ASYNC_DATABASE_URL, echo=True)

# Create a base class for declarative class definitions
Base = declarative_base()

# Define the wiki_summaries table as a class
class WikiSummary(Base):
    __tablename__ = 'wiki_summaries'
    uuid = Column(String, primary_key=True)
    url = Column(String, nullable=False)
    summary = Column(Text, nullable=False)
    creation_date = Column(DateTime,
                           default=dt.now(tz=None))

# Create the table in the database
async def init_db():
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)

# Create a session factory
async_session = sessionmaker(
    bind=engine,
    class_=AsyncSession,
    expire_on_commit=False
)

async def insert_wiki_summary(uuid, url, summary):
    async with async_session() as session:
        async with session.begin():
            try:
                # Create a new WikiSummary object
                new_summary = WikiSummary(
                    uuid=uuid,
                    url=url,
                    summary=summary
                )
                
                # Add the new object to the session
                session.add(new_summary)
                
                # Commit the transaction
                await session.commit()
                
                print("Data inserted successfully")
                
            except Exception as e:
                print(f"An error occurred: {e}")
                await session.rollback()

# Example usage
async def main():
    await init_db()
    await insert_wiki_summary("123e4567-e89b-12d3-a456-426614174004", "https://example.com", "This is a summary.")

# Run the example
await main()

2024-09-21 02:13:38,718 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-09-21 02:13:38,719 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-21 02:13:38,722 INFO sqlalchemy.engine.Engine select current_schema()
2024-09-21 02:13:38,723 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-21 02:13:38,725 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-09-21 02:13:38,726 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-21 02:13:38,728 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-21 02:13:38,731 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = $1::VARCHAR AND pg_catalog.pg_class.relkind = ANY (ARRAY[$2::VARCHAR, $3::VARCHAR, $4::VARCHAR, $5::VARCHAR, $6::VARCHAR]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != $7::VARCHAR
